# Purpose

Find a way to get data from Plugshare.com since they're not responding to my API access request. The comments and metadata from stations across different networks should be extremely useful in diagnosing electrical and non-electrical customer experience issues.

# Imports

In [1]:

%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import pandas as pd

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-2.0")
logger.info("TEST!")

2024-07-29_T20_01_11EDT: INFO (Notebook-2.0:L16) - TEST!


In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Trying out Folium

I hear it's a very capable high-quality mapping tool, so let's give it a whirl! We'll do everything initially with OpenStreetMap and open source routing to get a feel for things (may eventually switch to Google Routes API).

## Starting Point Map

In [3]:
import folium

WHITEHOUSE_COORDS = (38.8964457,-77.035921) # for funsies
BOSTON_COORDS = (42.3144474,-71.0526845)

start_point_map = folium.Map(location=WHITEHOUSE_COORDS, zoom_start=16, tiles='OpenStreetMap')

# Give it a marker at our start point
folium.Marker(WHITEHOUSE_COORDS, draggable=True).add_to(start_point_map)
start_point_map

In [130]:
from evlens.visualization.folium_tools import get_single_point

get_single_point(WHITEHOUSE_COORDS, 'White House')

## Visualizing a Route

In [4]:
import openrouteservice as ors
import os

client = ors.Client(key=os.getenv('ORS_API_KEY'))

In [131]:
type(route)

dict

In [133]:
from evlens.data.routing import get_openrouting_route

r = get_openrouting_route(WHITEHOUSE_COORDS, BOSTON_COORDS, reverse_coordinates=True)
len(r)

5459

In [5]:
source = WHITEHOUSE_COORDS
destination = BOSTON_COORDS

# Get directions between the two points
# Note: OpenRouteService expects (lng, lat) tuples
directions_coordinates = [tuple(reversed(source)), tuple(reversed(destination))]  
route = client.directions(coordinates=directions_coordinates, profile='driving-car', format='geojson')

# extracting only coordinates
routes_coords= [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]
routes_coords

[[38.896452, -77.033619],
 [38.897358, -77.033624],
 [38.89832, -77.033626],
 [38.898806, -77.033627],
 [38.898824, -77.033472],
 [38.899356, -77.031949],
 [38.899922, -77.030325],
 [38.899976, -77.030184],
 [38.900115, -77.029799],
 [38.900157, -77.029624],
 [38.900671, -77.028104],
 [38.900805, -77.028104],
 [38.901337, -77.028102],
 [38.901877, -77.028101],
 [38.902412, -77.028099],
 [38.902526, -77.028099],
 [38.902527, -77.027033],
 [38.903031, -77.027037],
 [38.903747, -77.02704],
 [38.903958, -77.027039],
 [38.904172, -77.027038],
 [38.904659, -77.027038],
 [38.905281, -77.027039],
 [38.905658, -77.027039],
 [38.906117, -77.02704],
 [38.907246, -77.027037],
 [38.907495, -77.027039],
 [38.907646, -77.027038],
 [38.90794, -77.02704],
 [38.90816, -77.027038],
 [38.908218, -77.027038],
 [38.908583, -77.027039],
 [38.908935, -77.027038],
 [38.909008, -77.027038],
 [38.909299, -77.02704],
 [38.909646, -77.027042],
 [38.910001, -77.027037],
 [38.910094, -77.027038],
 [38.910482, -77.02

In [6]:
route

{'type': 'FeatureCollection',
 'bbox': [-77.033627, 38.896452, -71.052537, 42.358271],
 'features': [{'bbox': [-77.033627, 38.896452, -71.052537, 42.358271],
   'type': 'Feature',
   'properties': {'segments': [{'distance': 729327.4,
      'duration': 31398.5,
      'steps': [{'distance': 261.7,
        'duration': 34.6,
        'type': 11,
        'instruction': 'Head north on 15th Street Northwest',
        'name': '15th Street Northwest',
        'way_points': [0, 3]},
       {'distance': 521.6,
        'duration': 53.7,
        'type': 1,
        'instruction': 'Turn right onto New York Avenue Northwest',
        'name': 'New York Avenue Northwest',
        'way_points': [3, 10]},
       {'distance': 206.3,
        'duration': 16.5,
        'type': 0,
        'instruction': 'Turn left onto 12th Street Northwest',
        'name': '12th Street Northwest',
        'way_points': [10, 15]},
       {'distance': 92.2,
        'duration': 3.9,
        'type': 1,
        'instruction': 'Tur

In [7]:
len(routes_coords)

5459

In [121]:
# Add our destination point
start_icon = folium.Icon(color='green', icon='fa-solid fa-play', prefix='fa')
end_icon = folium.Icon(color='red', icon='fa-solid fa-stop', prefix='fa')

map = folium.Map(location=WHITEHOUSE_COORDS, zoom_start=7)
folium.Marker(WHITEHOUSE_COORDS, popup='Washington, DC', icon=start_icon).add_to(map)
folium.Marker(BOSTON_COORDS, popup='Boston, MA', icon=end_icon).add_to(map)
map

In [138]:
from evlens.visualization.folium_tools import plot_route

plot_route(WHITEHOUSE_COORDS, BOSTON_COORDS, 'White House', 'Boston, MA', route_coordinates=routes_coords)

## Find stations near route

This is the simple version that uses NREL's nearest-to-route API. We'll eventually use our own approach and our own station data of course.

In [63]:
# Find NREL-supported charging stations near route
from evlens.data.nrel_api import AFDCStationsByRoute
from shapely.geometry import LineString

# Get WKT-structured route
route_linestring_wkt = str(LineString(route['features'][0]['geometry']['coordinates']))

afdc = AFDCStationsByRoute()
url, post_data = afdc._build_url(route_linestring_wkt, 5.0)
len(url)

70

In [151]:
df = afdc.get_stations_near_route(route_linestring_wkt, 1.0) # 0.25 miles from initial route
df.info()

2024-07-29_T23_49_57EDT: INFO (evlens.data.nrel_api:L190) - 63 total records found, comprised of 631 plugs


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 75 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   access_code                 63 non-null     object 
 1   access_days_time            63 non-null     object 
 2   access_detail_code          7 non-null      object 
 3   cards_accepted              30 non-null     object 
 4   date_last_confirmed         62 non-null     object 
 5   expected_date               0 non-null      object 
 6   fuel_type_code              63 non-null     object 
 7   groups_with_access_code     63 non-null     object 
 8   id                          63 non-null     int64  
 9   maximum_vehicle_class       30 non-null     object 
 10  open_date                   63 non-null     object 
 11  owner_type_code             63 non-null     object 
 12  restricted_access           28 non-null     object 
 13  status_code                 63 non-nu

In [156]:
from evlens.visualization.folium_tools import plot_route, add_stations_to_map

map = plot_route(WHITEHOUSE_COORDS, BOSTON_COORDS, 'White House', 'Boston, MA', route_coordinates=routes_coords)
map = add_stations_to_map(df, map, tooltip=True, networks_to_include='all')
map

2024-07-29_T23_56_52EDT: INFO (evlens.visualization.folium_tools:L123) - Counts of networks along the route: ev_network
Tesla         62
APPLEGREEN     1
Name: count, dtype: int64
